## NXOS交换机配置
* 用户名密码，设置权限为 network-admin
* feature bash-shell
* feature netconf
* feature restconf
* feature grpc
* feature stfp-server

In [1]:
import xmltodict
from ncclient import manager

In [2]:
NXOS_netconf_info = {"host": "192.168.1.65",
                     "port": 830,
                     "username": "netconf",
                     "password": "netconf@123",
                     "hostkey_verify": False}

In [3]:
with manager.connect(**NXOS_netconf_info) as device:
    for cap in device.server_capabilities:
        for capability in device.server_capabilities:
            print(capability.split('?')[0]) 

urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:writable-running:1.0
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:candidate:1.0
urn:ietf:params:netconf:capability:validate:1.1
urn:ietf:params:netconf:capability:confirmed-commit:1.1
urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:netconf:capability:interleave:1.0
urn:ietf:params:netconf:capability:with-defaults:1.0
http://cisco.com/ns/yang/cisco-nx-os-device
urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:writable-running:1.0
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:candidate:1.0
urn:ietf:params:netconf:capability:validate:1.1
urn:ietf:params:netconf:capability:confirmed-commit:1.1
urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:netconf:capability:interleave:1.0
urn:ietf:params:netconf:capability:

## 获取设备序列号

In [4]:
serial_filter = """
 <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
   <serial/>
 </System>
 """

In [5]:
with manager.connect(**NXOS_netconf_info) as device:
    netconf_response = device.get(('subtree', serial_filter))

In [6]:
netconf_response

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:201622a9-9afc-4e8a-baaa-a179af2ebdfe">
    <data>
        <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
            <serial>9LR0TGZ2MHQ</serial>
        </System>
    </data>
</rpc-reply>

In [7]:
Serial = xmltodict.parse(netconf_response.xml)['rpc-reply']['data']['System']['serial']
Serial

'9LR0TGZ2MHQ'

## 添加环回口

In [22]:
loopback_info = {"id": "99", "ip": "10.99.99.1/24"}

In [12]:
loopback_add = """<config>
<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    <intf-items>
        <lb-items>
            <LbRtdIf-list>
                <id>lo{id}</id>
                <adminSt>up</adminSt>
                <descr>Full intf config via NETCONF</descr>
            </LbRtdIf-list>
        </lb-items>
    </intf-items>
    <ipv4-items>
        <inst-items>
            <dom-items>
                <Dom-list>
                    <name>default</name>
                    <if-items>
                        <If-list>
                            <id>lo{id}</id>
                            <addr-items>
                                <Addr-list>
                                    <addr>{ip}</addr>
                                </Addr-list>
                            </addr-items>
                        </If-list>
                    </if-items>
                </Dom-list>
            </dom-items>
        </inst-items>
    </ipv4-items>
</System>
</config>""".format(id = loopback_info["id"], ip = loopback_info["ip"])

In [13]:
with manager.connect(**NXOS_netconf_info) as device:
    print(device.edit_config(target='running', config=loopback_add))

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:b4e02349-2b12-4410-8e2f-7f07ba78b00d">
    <ok/>
</rpc-reply>



## 查询环回口

In [14]:
loopback_filter = """
<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    <intf-items>
        <lb-items/>
    </intf-items>
</System>
"""
ip_interface_filter = """
<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
<ipv4-items>
    <inst-items>
        <dom-items>
            <Dom-list>
                <name>default</name>
                <if-items>
                    <If-list/>
                </if-items>
            </Dom-list>
        </dom-items>
    </inst-items>
</ipv4-items>
</System>
"""

In [15]:
with manager.connect(**NXOS_netconf_info) as device:
    loopback_response = device.get(('subtree', loopback_filter))
    ip_interface_response = device.get(('subtree', ip_interface_filter))

In [16]:
loop_back_data = xmltodict.parse(loopback_response.xml)['rpc-reply']['data']['System']["intf-items"]["lb-items"]["LbRtdIf-list"]
loop_back_data

OrderedDict([('id', 'lo99'),
             ('adminSt', 'up'),
             ('descr', 'Full intf config via NETCONF'),
             ('lbrtdif-items',
              OrderedDict([('currErrIndex', '4294967295'),
                           ('ifIndex', '335544419'),
                           ('iod', '5'),
                           ('lastErrors', '0,0,0,0'),
                           ('operBitset', None),
                           ('operDescr', 'Full intf config via NETCONF'),
                           ('operMtu', '1500'),
                           ('operSt', 'up'),
                           ('operStQual', 'none')])),
             ('linkLog', 'default'),
             ('rtvrfMbr-items',
              OrderedDict([('tDn',
                            "/System/inst-items/Inst-list[name='default']")]))])

In [20]:
print(f"环回接口{loop_back_data['id']}状态为{loop_back_data['adminSt']}")

环回接口lo99状态为up


In [19]:
ip_if_data = xmltodict.parse(ip_interface_response.xml)['rpc-reply']['data']['System']["ipv4-items"]["inst-items"]["dom-items"]["Dom-list"]["if-items"]["If-list"]
ip_if_data

OrderedDict([('id', 'lo99'),
             ('addr-items',
              OrderedDict([('Addr-list',
                            OrderedDict([('addr', '10.99.99.1/24'),
                                         ('operSt', 'up'),
                                         ('operStQual', 'unspecified'),
                                         ('pref', '0'),
                                         ('tag', '0'),
                                         ('type', 'primary')]))])),
             ('directedBroadcast', 'disabled'),
             ('dropGlean', 'disabled'),
             ('forward', 'disabled'),
             ('operSt', 'up'),
             ('operStQual', 'unspecified'),
             ('urpf', 'disabled')])

In [21]:
print(f"环回接口{ip_if_data['id']}配置了IP地址：{ip_if_data['addr-items']['Addr-list']['addr']}")

环回接口lo99配置了IP地址：10.99.99.1/24


## 删除环回口

In [23]:
loopback_remove = """
<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <intf-items>
            <lb-items>
                <LbRtdIf-list operation="remove">
                    <id>lo{id}</id>
                </LbRtdIf-list>
            </lb-items>
        </intf-items>
    </System>
</config>""".format(id = loopback_info["id"], ip = loopback_info["ip"])

In [24]:
with manager.connect(**NXOS_netconf_info) as device:
    print(device.edit_config(target='running', config=loopback_remove))

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:d5a09e90-2b6c-4c1a-a196-831dd737e08c">
    <ok/>
</rpc-reply>



## 再次查看环回口

In [25]:
with manager.connect(**NXOS_netconf_info) as device:
    loopback_response = device.get(('subtree', loopback_filter))
    ip_interface_response = device.get(('subtree', ip_interface_filter))

In [26]:
loopback_response

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:0180cf9c-aad3-49cf-adcb-8cdc9641ae33">
    <data>
        <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device"/>
    </data>
</rpc-reply>

In [27]:
ip_interface_response

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:51a86cee-89e8-4e41-8201-b0995e57b0dd">
    <data>
        <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
            <ipv4-items>
                <inst-items>
                    <dom-items>
                        <Dom-list>
                            <name>default</name>
                        </Dom-list>
                    </dom-items>
                </inst-items>
            </ipv4-items>
        </System>
    </data>
</rpc-reply>